# Credit Risk Resampling Techniques

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

# Read the CSV and Perform Basic Data Cleaning

In [3]:
columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [4]:
# Load the data
file_path = Path('data/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()
df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,20000.0,17.19%,499.10,RENT,47000.0,Source Verified,Mar-2019,Issued,n,14.02,...,98.0,12.5,0.0,0.0,75824.0,31546.0,33800.0,21524.0,N,N
1,21225.0,14.74%,502.05,MORTGAGE,225000.0,Not Verified,Mar-2019,Issued,n,16.80,...,100.0,50.0,1.0,0.0,747075.0,209426.0,53500.0,128175.0,N,N
2,5000.0,17.97%,180.69,MORTGAGE,62000.0,Not Verified,Mar-2019,Issued,n,19.82,...,66.7,50.0,0.0,0.0,255738.0,31615.0,9400.0,39938.0,N,N
3,20000.0,8.19%,628.49,MORTGAGE,200000.0,Not Verified,Mar-2019,Issued,n,22.66,...,95.7,22.2,0.0,0.0,448069.0,84744.0,49400.0,105180.0,N,N
4,12000.0,15.57%,289.09,MORTGAGE,49000.0,Source Verified,Mar-2019,Issued,n,13.47,...,86.1,14.3,0.0,0.0,189260.0,106025.0,24400.0,68860.0,N,N


In [5]:
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [6]:
df.columns

Index(['loan_amnt', 'int_rate', 'installment', 'home_ownership', 'annual_inc',
       'verification_status', 'issue_d', 'loan_status', 'pymnt_plan', 'dti',
       'delinq_2yrs', 'inq_last_6mths', 'open_acc', 'pub_rec', 'revol_bal',
       'total_acc', 'initial_list_status', 'out_prncp', 'out_prncp_inv',
       'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int',
       'total_rec_late_fee', 'recoveries', 'collection_recovery_fee',
       'last_pymnt_amnt', 'next_pymnt_d', 'collections_12_mths_ex_med',
       'policy_code', 'application_type', 'acc_now_delinq', 'tot_coll_amt',
       'tot_cur_bal', 'open_acc_6m', 'open_act_il', 'open_il_12m',
       'open_il_24m', 'mths_since_rcnt_il', 'total_bal_il', 'il_util',
       'open_rv_12m', 'open_rv_24m', 'max_bal_bc', 'all_util',
       'total_rev_hi_lim', 'inq_fi', 'total_cu_tl', 'inq_last_12m',
       'acc_open_past_24mths', 'avg_cur_bal', 'bc_open_to_buy', 'bc_util',
       'chargeoff_within_12_mths', 'delinq_amnt', 'mo_si

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68817 entries, 0 to 68816
Data columns (total 86 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   loan_amnt                   68817 non-null  float64
 1   int_rate                    68817 non-null  float64
 2   installment                 68817 non-null  float64
 3   home_ownership              68817 non-null  object 
 4   annual_inc                  68817 non-null  float64
 5   verification_status         68817 non-null  object 
 6   issue_d                     68817 non-null  object 
 7   loan_status                 68817 non-null  object 
 8   pymnt_plan                  68817 non-null  object 
 9   dti                         68817 non-null  float64
 10  delinq_2yrs                 68817 non-null  float64
 11  inq_last_6mths              68817 non-null  float64
 12  open_acc                    68817 non-null  float64
 13  pub_rec                     688

# Split the Data into Training and Testing

In [8]:
# Create our features
df_encode = pd.get_dummies(df, columns=["home_ownership","verification_status", "issue_d", "pymnt_plan","initial_list_status","next_pymnt_d", "application_type","hardship_flag", "debt_settlement_flag"])
X = df_encode.drop(columns=['loan_status'], axis=1)

# Create our target
y = df['loan_status']

In [9]:
pd.set_option('display.max_rows', None)
X.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
loan_amnt,68817.0,16677.594562,10277.348590,1000.00,9000.0000,15000.000,24000.0000,4.000000e+04
int_rate,68817.0,0.127718,0.048130,0.06,0.0881,0.118,0.1557,3.084000e-01
installment,68817.0,480.652863,288.062432,30.89,265.7300,404.560,648.1000,1.676230e+03
annual_inc,68817.0,88213.705210,115579.958439,40.00,50000.0000,73000.000,104000.0000,8.797500e+06
dti,68817.0,21.778153,20.199244,0.00,13.8900,19.760,26.6600,9.990000e+02
delinq_2yrs,68817.0,0.217766,0.718367,0.00,0.0000,0.000,0.0000,1.800000e+01
inq_last_6mths,68817.0,0.497697,0.758122,0.00,0.0000,0.000,1.0000,5.000000e+00
open_acc,68817.0,12.587340,6.022869,2.00,8.0000,11.000,16.0000,7.200000e+01
pub_rec,68817.0,0.126030,0.336797,0.00,0.0000,0.000,0.0000,4.000000e+00
revol_bal,68817.0,17604.142828,21835.880400,0.00,6293.0000,12068.000,21735.0000,5.871910e+05


In [10]:
# Check the balance of our target values
y.value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [11]:
# import sklearn train_test_split
from sklearn.model_selection import train_test_split

# Split the dataset into training and testing subsets 
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1 ,
                                                   stratify = y)
# Counter module confirms the imbalance in the training set.
Counter(y_train)

Counter({'low_risk': 51352, 'high_risk': 260})

# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [12]:
# Resample the training data with the RandomOversampler
from imblearn.over_sampling import RandomOverSampler
from termcolor import colored

ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

Counter(y_resampled)

Counter({'low_risk': 51352, 'high_risk': 51352})

In [13]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)


LogisticRegression(random_state=1)

In [14]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
from imblearn.metrics import classification_report_imbalanced
from imblearn.metrics import classification_report_imbalanced

## make predictions and calucate balanced accuracy score
y_pred = model.predict(X_test)
bac_random = balanced_accuracy_score(y_test, y_pred)
print('The balanced accuracy score for naive random oversampling is', bac_random, '\n')

# Display the confusion matrix
from sklearn.metrics import confusion_matrix
cm_random = confusion_matrix(y_test, y_pred)
print('The confusion matrix for naive random oversampling is \n', cm_random, '\n')

# Print the imbalanced classification report
imb_classRep_random = classification_report_imbalanced(y_test, y_pred)
print('The imbalanced classification report for naive random oversampling is \n', imb_classRep_random)

The balanced accuracy score for naive random oversampling is 0.6612700484668286 

The confusion matrix for naive random oversampling is 
 [[   57    30]
 [ 5694 11424]] 

The imbalanced classification report for naive random oversampling is 
                    pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.66      0.67      0.02      0.66      0.44        87
   low_risk       1.00      0.67      0.66      0.80      0.66      0.44     17118

avg / total       0.99      0.67      0.66      0.80      0.66      0.44     17205



### SMOTE Oversampling

In [15]:
# Resample the training data with SMOTE
from imblearn.over_sampling import SMOTE
X_resampled, y_resampled = SMOTE(random_state=1,sampling_strategy='auto').fit_resample(
X_train, y_train)
Counter(y_resampled)

Counter({'low_risk': 51352, 'high_risk': 51352})

In [16]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [17]:
## make predictions and calucate balanced accuracy score
y_pred = model.predict(X_test)
bac_SMOTE = balanced_accuracy_score(y_test, y_pred)
print('The balanced accuracy score for SMOTE oversampling is', bac_SMOTE, '\n')
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
cm_SMOTE = confusion_matrix(y_test, y_pred)
print('The confusion matrix for SMOTE oversampling is \n', cm_SMOTE, '\n')

# Print the imbalanced classification report
imb_classRep_SMOTE = classification_report_imbalanced(y_test, y_pred)
print('The imbalanced classification report for SMOTE oversampling is \n', imb_classRep_SMOTE)

The balanced accuracy score for SMOTE oversampling is 0.6303296388959394 

The confusion matrix for SMOTE oversampling is 
 [[   54    33]
 [ 6163 10955]] 

The imbalanced classification report for SMOTE oversampling is 
                    pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.62      0.64      0.02      0.63      0.40        87
   low_risk       1.00      0.64      0.62      0.78      0.63      0.40     17118

avg / total       0.99      0.64      0.62      0.78      0.63      0.40     17205



# Undersampling


In [18]:
# Resample the data using the ClusterCentroids resampler
from imblearn.under_sampling import RandomUnderSampler
ros = RandomUnderSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({'high_risk': 260, 'low_risk': 260})

In [19]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [20]:
## make predictions and calucate balanced accuracy score
y_pred = model.predict(X_test)
bac_undersample = balanced_accuracy_score(y_test, y_pred)
print('The balanced accuracy score for clustercentroids underresampling is', bac_undersample, '\n')
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
cm_undersample = confusion_matrix(y_test, y_pred)
print('The confusion matrix for clustercentroids underresampling is \n', cm_undersample, '\n')

# Print the imbalanced classification report
imb_classRep_undersample = classification_report_imbalanced(y_test, y_pred)
print('The imbalanced classification report for clustercentroids underresampling is \n', imb_classRep_undersample)

The balanced accuracy score for clustercentroids underresampling is 0.590320332297924 

The confusion matrix for clustercentroids underresampling is 
 [[  53   34]
 [7336 9782]] 

The imbalanced classification report for clustercentroids underresampling is 
                    pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.61      0.57      0.01      0.59      0.35        87
   low_risk       1.00      0.57      0.61      0.73      0.59      0.35     17118

avg / total       0.99      0.57      0.61      0.72      0.59      0.35     17205



# Combination (Over and Under, SMOTEENN) Sampling


In [21]:
# Resample the training data with SMOTEENN
from imblearn.combine import SMOTEENN
smote_enn = SMOTEENN(random_state=0)
X_resampled, y_resampled = smote_enn.fit_resample(X, y)
Counter(y_resampled)

Counter({'high_risk': 68460, 'low_risk': 62011})

In [22]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [23]:
## make predictions and calucate balanced accuracy score
y_pred = model.predict(X_test)
bac_SMOTEENN = balanced_accuracy_score(y_test, y_pred)
print('The balanced accuracy score for SMOTEENN combination sampling is', bac_SMOTEENN, '\n')
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
cm_SMOTEENN = confusion_matrix(y_test, y_pred)
print('The confusion matrix for SMOTEENN combination sampling is \n', cm_SMOTEENN , '\n')

# Print the imbalanced classification report
imb_classRep_SMOTEENN = classification_report_imbalanced(y_test, y_pred)
print('The imbalanced classification report for SMOTEENN combination sampling is \n', imb_classRep_SMOTEENN)

The balanced accuracy score for SMOTEENN combination sampling is 0.6375241226214794 

The confusion matrix for SMOTEENN combination sampling is 
 [[  61   26]
 [7294 9824]] 

The imbalanced classification report for SMOTEENN combination sampling is 
                    pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.70      0.57      0.02      0.63      0.41        87
   low_risk       1.00      0.57      0.70      0.73      0.63      0.40     17118

avg / total       0.99      0.57      0.70      0.72      0.63      0.40     17205



In [24]:
# Print the all
from termcolor import colored

# naive random oversampling
print(colored('Balanced accuracy score \n', 'blue', attrs=['bold']))

print('Naive random oversampling : ', bac_random)
print('SMOTE oversampling : ', bac_SMOTE)
print('Clustercentroids underresampling : ', bac_undersample)
print('SMOTEENN combination sampling : ', bac_SMOTEENN, '\n\n')

print(colored('Confusion matrix \n', 'blue', attrs=['bold']))

print('Naive random oversampling : \n', cm_random, '\n')
print('SMOTE oversampling is : \n ', cm_SMOTE, '\n')
print('Clustercentroids underresampling : \n ', cm_undersample, '\n')
print('SMOTEENN combination sampling : \n', cm_SMOTEENN , '\n\n')


print(colored('Imbalanced classification report \n', 'blue', attrs=['bold']))

print('Naive random oversampling : \n', imb_classRep_random, '\n')
print('SMOTE oversampling is : \n', imb_classRep_SMOTE, '\n')
print('Clustercentroids underresampling : \n', imb_classRep_undersample, '\n')
print('SMOTEENN combination sampling : \n', imb_classRep_SMOTEENN, '\n')


Balanced accuracy score 

Naive random oversampling :  0.6612700484668286
SMOTE oversampling :  0.6303296388959394
Clustercentroids underresampling :  0.590320332297924
SMOTEENN combination sampling :  0.6375241226214794 


Confusion matrix 

Naive random oversampling : 
 [[   57    30]
 [ 5694 11424]] 

SMOTE oversampling is : 
  [[   54    33]
 [ 6163 10955]] 

Clustercentroids underresampling : 
  [[  53   34]
 [7336 9782]] 

SMOTEENN combination sampling : 
 [[  61   26]
 [7294 9824]] 


Imbalanced classification report 

Naive random oversampling : 
                    pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.66      0.67      0.02      0.66      0.44        87
   low_risk       1.00      0.67      0.66      0.80      0.66      0.44     17118

avg / total       0.99      0.67      0.66      0.80      0.66      0.44     17205
 

SMOTE oversampling is : 
                    pre       rec       spe        f1       geo       iba   